In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import nltk
from gensim.models import Word2Vec
from gensim.models import doc2vec
from collections import Counter
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.stem.snowball import SnowballStemmer
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold, train_test_split

In [2]:
def count(text):
    return len(text.split())
def high_score(i):
    if i>3:
        return 1
    else: return 0

In [3]:
swords = stopwords.words('russian')

In [4]:
stem = SnowballStemmer('russian')

In [5]:
df = pd.read_csv('./wb_school_task_2.csv.gzip', compression='gzip')

In [6]:
df.head()

,id1,id2,id3,text,f1,f2,f3,f4,f5,f6,f7,f8,label
0,7596126584852021591,173777575861152844,18254136755182295358,"Хотела купить на замену старых,по итогу эти у...",1,1,1.0,11,13,4.272727,18,36,0
1,5327406586753254371,14366783259208998381,2324030737335224220,Запах по сравнению с обычным DOVE никакой. Оно...,10,6,2.7,46,4,4.434783,90,109,1
2,2636757786779776109,16985421652518589249,4904562693381133981,"Кабель подошёл, хорошо работает.",4,4,4.0,14,0,4.500000,4,6,0
3,15432976385014516263,2629180387521298831,13541353053200353836,"Восторг! Очень приятный аромат, держится долго...",6,6,5.0,374,30,4.772727,14,15,1
4,11933982800034911890,14531191561111600318,9121890650636487522,Визуально все хорошо. Не пробовали. Купили в п...,19,18,5.0,6,4,5.000000,26,33,0


In [7]:
text = df.text

In [8]:
text[0]

'Хотела купить на замену старых,по итогу эти  ужаснотупые ,не удобные ,рвут кутикулу ,а не режут.'

# Первичная обработка

In [9]:
tokenized_sw_p = pd.Series([nltk.word_tokenize(text[i], language='russian') for i in range(0, len(text))])

In [10]:
tokenized_sw_p.head()

0    [Хотела, купить, на, замену, старых, ,, по, ит...
1    [Запах, по, сравнению, с, обычным, DOVE, никак...
2            [Кабель, подошёл, ,, хорошо, работает, .]
3    [Восторг, !, Очень, приятный, аромат, ,, держи...
4    [Визуально, все, хорошо, ., Не, пробовали, ., ...
dtype: object

In [11]:
stemmed = []
for i in range(0, len(tokenized_sw_p)):
    stemmed.append([stem.stem(token) for token in tokenized_sw_p[i] if token not in swords and token not in punctuation])
stemmed = pd.Series(stemmed)

In [12]:
del tokenized_sw_p
del text

In [13]:
for i in range(0, len(stemmed)):
    stemmed[i] = " ".join(stemmed[i])

In [14]:
stemmed_tok = pd.Series([nltk.word_tokenize(stemmed[i], language='russian') for i in range(0, len(stemmed))])

In [15]:
df.label[1]

1

In [16]:
stemmed_tag=[]
for i in range(0, len(stemmed_tok)):
    stemmed_tag.append(doc2vec.TaggedDocument(stemmed_tok[i], [i]))

In [17]:
wvec = Word2Vec(vector_size=100, min_count=1, workers = -1)

In [18]:
wvec.build_vocab(stemmed_tok)

In [19]:
wvec.train(stemmed_tok, total_examples=wvec.corpus_count, epochs=30)

(0, 0)

In [20]:
docvec = doc2vec.Doc2Vec(vector_size=100, min_count=1, workers = -1)

In [21]:
docvec.build_vocab(stemmed_tag)

In [22]:
docvec.train(stemmed_tag, total_examples=docvec.corpus_count, epochs=30)

In [23]:
av=[]
for i in range(0, len(stemmed_tok)):
    l = np.array([wvec.wv[tok] for tok in stemmed_tok[i]])
    av.append((np.average(a=l,  axis = 1, weights=docvec[i])))
av = pd.Series(av) 

In [24]:
X = [[]]*len(stemmed_tok)
for i in range(0, len(stemmed_tok)):
    X[i]= np.append(X[i], av[i].min())
    X[i]= np.append(X[i], av[i].max())

In [25]:
del av

In [26]:
y = df.label

In [27]:
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_tr, y_tr, test_size=0.3, random_state=42)
del X_tr, y_tr

In [28]:
clf = LogisticRegression()